In [ ]:
import os, sys
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import datetime
import pickle
from lightgbm import *
from sklearn.model_selection import *
from sklearn.ensemble import *
import scipy
from xgboost import *
from catboost import *

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%y')

In [ ]:
trainDF = pd.read_csv('dataset/train.csv', parse_dates=['booking_date', 'checkin_date', 'checkout_date'], date_parser=dateparse)
print('Done loading training data')
testDF = pd.read_csv('dataset/test.csv', parse_dates=['booking_date', 'checkin_date', 'checkout_date'], date_parser=dateparse)
print('Done loading testing data')

In [ ]:
dir(trainDF['checkin_date'].dt)
newFeatures = ['day', 'dayofweek', 'dayofyear', 'days_in_month', 'is_leap_year', 'is_month_end',
 'is_month_start', 'is_quarter_end', 'is_quarter_start', 'is_year_end','is_year_start','month',
 'quarter','weekofyear', 'year']


In [ ]:
########################checkin, checkout and booking dates##########################################

for feature in newFeatures:
    trainDF['xxx_checkin_'+feature] = getattr(trainDF['checkin_date'].dt, feature)
for feature in newFeatures:
    trainDF['xxx_checkout_'+feature] = getattr(trainDF['checkout_date'].dt, feature)
for feature in newFeatures:
    trainDF['xxx_booking_'+feature] = getattr(trainDF['booking_date'].dt, feature)
    
for feature in newFeatures:
    testDF['xxx_checkin_'+feature] = getattr(testDF['checkin_date'].dt, feature)
for feature in newFeatures:
    testDF['xxx_checkout_'+feature] = getattr(testDF['checkout_date'].dt, feature)
for feature in newFeatures:
    testDF['xxx_booking_'+feature] = getattr(testDF['booking_date'].dt, feature)
    

    

In [ ]:
########################checkin, checkout and booking dates##########################################

trainDF['xxx_checkin_boarding_diff'] = (trainDF['checkin_date']-trainDF['booking_date']).dt.days
testDF['xxx_checkin_boarding_diff'] = (testDF['checkin_date']-testDF['booking_date']).dt.days

trainDF['xxx_checkout_checkin_diff'] = (trainDF['checkout_date']-trainDF['checkin_date']).dt.days
testDF['xxx_checkout_checkin_diff'] = (testDF['checkout_date']-testDF['checkin_date']).dt.days


trainDF['xxx_new_checkin'] = trainDF['checkin_date'].dt.date-datetime.date(2012, 3, 21)
testDF['xxx_new_checkin'] = testDF['checkin_date'].dt.date-datetime.date(2012, 3, 21)

trainDF['xxx_new_checkin'] = trainDF['xxx_new_checkin'].dt.days
testDF['xxx_new_checkin'] = testDF['xxx_new_checkin'].dt.days

trainDF['xxx_stayed_minus_roomnights'] = trainDF['roomnights']-trainDF['xxx_checkout_checkin_diff']
testDF['xxx_stayed_minus_roomnights'] = testDF['roomnights']-testDF['xxx_checkout_checkin_diff']




In [ ]:
del trainDF['checkin_date']
del testDF['checkin_date']
del trainDF['checkout_date']
del testDF['checkout_date']
del trainDF['booking_date']
del testDF['booking_date']

In [ ]:
##################################main_product_code####################################################
trainDF['main_product_code'] = trainDF['main_product_code'].replace(7, 0)
testDF['main_product_code'] = testDF['main_product_code'].replace(7, 0)

#pass

In [ ]:
##################################numberofadults, numberofchildren#####################################

trainDF['xxx_diff_people'] = -trainDF['numberofchildren']-trainDF['numberofadults']+trainDF['total_pax'] 
testDF['xxx_diff_people'] = -testDF['numberofchildren']-testDF['numberofadults']+testDF['total_pax']

trainDF['xxx_child_plus_adults'] = trainDF['numberofchildren'] + trainDF['numberofadults']
testDF['xxx_child_plus_adults'] = testDF['numberofchildren'] + testDF['numberofadults']

trainDF['xxx_child_fraction'] = trainDF['numberofchildren']/trainDF['xxx_child_plus_adults']
testDF['xxx_child_fraction'] = testDF['numberofchildren']/testDF['xxx_child_plus_adults']

trainDF['xxx_adult_fraction'] = trainDF['numberofadults']/trainDF['xxx_child_plus_adults']
testDF['xxx_adult_fraction'] = testDF['numberofadults']/testDF['xxx_child_plus_adults']

trainDF['xxx_ignore_pax_or_ch_ad'] = (trainDF['xxx_child_plus_adults']==trainDF['total_pax'])
testDF['xxx_ignore_pax_or_ch_ad'] = (testDF['xxx_child_plus_adults']==testDF['total_pax'])

trainDF['xxx_adult_fraction'].fillna(0, inplace=True)
testDF['xxx_adult_fraction'].fillna(0, inplace=True)
trainDF['xxx_child_fraction'].fillna(0, inplace=True)
testDF['xxx_child_fraction'].fillna(0, inplace=True)

In [ ]:
#################################persontravellingid###################################################

trainDF['xxx_persontravellingid'] = trainDF['persontravellingid'].replace([45, 47, 46, 4752, 4753, 4995], [0, 1, 2, 3, 4, 5])
testDF['xxx_persontravellingid'] = testDF['persontravellingid'].replace([45, 47, 46, 4752, 4753, 4995], [0, 1, 2, 3, 4, 5])

del trainDF['persontravellingid']
del testDF['persontravellingid']

In [ ]:
##################################season_holidayed_code###############################################
trainDF['season_holidayed_code'] = trainDF['season_holidayed_code'].fillna(0)
testDF['season_holidayed_code'] = testDF['season_holidayed_code'].fillna(0)


#pass

In [ ]:
###################################state_code_residence###############################################

trainDF['state_code_residence'] = trainDF['state_code_residence'].fillna(0)
testDF['state_code_residence'] = testDF['state_code_residence'].fillna(0)

In [ ]:
#############################member_age_buckets#######################################################

trainDF['xxx_member_age_buckets'] = trainDF['member_age_buckets'].replace(['D', 'E', 'F', 'C', 'H', 'G', 'B', 'I', 'A', 'J'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
testDF['xxx_member_age_buckets'] = testDF['member_age_buckets'].replace(['D', 'E', 'F', 'C', 'H', 'G', 'B', 'I', 'A', 'J'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

del trainDF['member_age_buckets']
del testDF['member_age_buckets']

In [ ]:
###########################################memberid###################################################

memberTrain = trainDF['memberid'].value_counts()
memberTest = testDF['memberid'].value_counts()

memberFeatureTrain = [0]*len(trainDF)
memberFeatureTest = [0]*len(testDF)

for i in range(len(trainDF)):
    memberFeatureTrain[i] = memberTrain[trainDF.loc[i, 'memberid']]

for i in range(len(testDF)):
    memberFeatureTest[i] = memberTest[testDF.loc[i, 'memberid']]
  
trainDF['xxx_member_feature'] = memberFeatureTrain
testDF['xxx_member_feature'] = memberFeatureTest

# del trainDF['memberid']
# del testDF['memberid']

In [ ]:
########################################cluster_code##################################################

trainDF['xxx_cluster'] = trainDF['cluster_code'].replace(['F', 'E', 'D', 'C', 'B', 'A'], [5, 4, 3, 2, 1, 0])
testDF['xxx_cluster'] = testDF['cluster_code'].replace(['F', 'E', 'D', 'C', 'B', 'A'], [5, 4, 3, 2, 1, 0])

del trainDF['cluster_code']
del testDF['cluster_code']

In [ ]:
########################################reservationstatusid_code##################################################

trainDF['xxx_reservationstatusid_code'] = trainDF['reservationstatusid_code'].replace(['A', 'B', 'C', 'D'], [0, 1, 2, 0])
testDF['xxx_reservationstatusid_code'] = testDF['reservationstatusid_code'].replace(['A', 'B', 'C', 'D'], [0, 1, 2, 0])

del trainDF['reservationstatusid_code']
del testDF['reservationstatusid_code']

In [ ]:
resortCount=trainDF['resort_id'].value_counts()

In [ ]:
###############################################resort_id##############################################

trainDF['xxx_resort_code'] = trainDF['resort_id'].replace(resortCount.index.tolist(), list(range(0, 32)))
print("Done with training data")
testDF['xxx_resort_code'] = testDF['resort_id'].replace(resortCount.index.tolist(), list(range(0, 32)))

del trainDF['resort_id']
del testDF['resort_id']

In [ ]:
cf = [
 'channel_code',
 'main_product_code',
 'resort_region_code',
 'resort_type_code',
 'room_type_booked_code',
 'season_holidayed_code',
 'state_code_residence',
 'state_code_resort',
 'booking_type_code',
 'xxx_ignore_pax_or_ch_ad',
 'xxx_persontravellingid',
 'xxx_member_age_buckets',
 'xxx_cluster',
 'xxx_reservationstatusid_code',
 'xxx_resort_code']

nf = [
 'numberofadults',
 'numberofchildren',
 'roomnights',
 'total_pax',
 'xxx_checkin_boarding_diff',
 'xxx_checkout_checkin_diff',
 'xxx_new_checkin',
 'xxx_stayed_minus_roomnights',
 'xxx_xcheckin_nhf',
 'xxx_diff_people',
 'xxx_child_plus_adults',
 'xxx_child_fraction',
 'xxx_adult_fraction']

In [ ]:
y = np.array(trainDF['amount_spent_per_room_night_scaled'].values)
testIds = np.array(testDF['reservation_id'].values)
groups = np.array(trainDF['memberid'].values)

In [ ]:
del trainDF['reservation_id']
del trainDF['amount_spent_per_room_night_scaled']
del testDF['reservation_id']
del trainDF['memberid']
del testDF['memberid']

In [ ]:
X = np.array(trainDF.values)
XTest = np.array(testDF.values)
featureColumns = np.array(trainDF.columns)

In [ ]:
print(X.shape, y.shape, XTest.shape)

In [ ]:
pickle.dump([X, y, XTest, testIds, featureColumns, groups], open('freshPickles/cleanExperimentDataF=74.pk', 'wb'))

In [ ]:
catFeatures = [0, 1, 4, 5, 6, 8, 9, 10, 12, 67, 68, 69, 71, 72, 73]

cfc = []
for z in catFeatures:
    cfc.append(featureColumns[z])
    print(featureColumns[z])

In [ ]:
model = CatBoostRegressor(cat_features=catFeatures)
scores = cross_val_score(model, X, y, cv=gfCV10, verbose=100, scoring='neg_mean_squared_error')

print(scores)
print(np.mean(scores))
print(np.sqrt(-1*scores))
print(np.mean(np.sqrt(-1*scores)))

In [ ]:
totalDF = pd.DataFrame(np.vstack((trainDF.values, testDF.values)), columns=trainDF.columns)

totalDF = pd.get_dummies(totalDF, columns=catFeatures)

trainDFdum = totalDF.iloc[:len(trainDF)]
testDFdum = totalDF.iloc[len(trainDF):]

In [ ]:
X = trainDFdum.values
XTest = testDFdum.values
featureColumns = trainDFdum.columns

In [ ]:
pickle.dump([X, y, XTest, testIds, featureColumns, groups], open('freshPickles/dummyCleanExperimentData.pk', 'wb'))